In [1]:
from transformers import pipeline

c:\Users\ifedi\Documents\Applied AI and ML\Session 2\Playground\venv\my_pytorch_cpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ifedi\Documents\Applied AI and ML\Session 2\Playground\venv\my_pytorch_cpu\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\ifedi\Documents\Applied AI and ML\Session 2\Playground\venv\my_pytorch_cpu\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvi

In [2]:
from transformers import pipeline, WhisperForConditionalGeneration, WhisperTokenizer, WhisperFeatureExtractor

# Load model without forced_decoder_ids
model_path = "C:/Users/ifedi/Documents/Applied AI and ML/Session 2/INFO8665 - Projects in Machine Learning/SageCare-2.0/dev/models/whisper-medical-v2-final"

model = WhisperForConditionalGeneration.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)


model.generation_config.forced_decoder_ids = None  # Disable forced IDs



In [3]:
# 3. Create pipeline with ALL components
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=tokenizer,
    feature_extractor=feature_extractor,
    device="cpu"  # or "cuda" if using GPU
)

# Transcribe audio
audio_path = "C:/Users/ifedi/Documents/Applied AI and ML/Session 2/INFO8665 - Projects in Machine Learning/SageCare-2.0/data-collection/unitedsynmed_small/audio/test/drug-brand-en-us-female-1beb8df1-473e-4017-9f19-fe8c88afa88d.wav"

result = pipe(audio_path)
print(result["text"])

Device set to use cpu
c:\Users\ifedi\Documents\Applied AI and ML\Session 2\Playground\venv\my_pytorch_cpu\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these value

 Make sure to follow the dosage instructions while taking APTOFRESH.                                                                                                                                                                                                                                                                                                                                                                                                                                            


In [ ]:
audio_path = "C:/Users/ifedi/Documents/Applied AI and ML/Session 2/INFO8665 - Projects in Machine Learning/SageCare-2.0/data-collection/unitedsynmed_small/audio/train/drug-brand-en-us-female-0d0979f3-3519-4e70-beae-339cda2b254a.wav"

result = pipe(audio_path)
print(result["text"])

In [ ]:
import onnxruntime as ort

In [ ]:
import numpy as np


In [ ]:
from transformers import WhisperProcessor

# Load ONNX model
sess = ort.InferenceSession("C:/Users/ifedi/Documents/Applied AI and ML/Session 2/INFO8665 - Projects in Machine Learning/SageCare-2.0/dev/models/whisper_medical.onnx")

# Load processor (for tokenization & feature extraction)
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# Manual inference logic needed...
def transcribe_audio(audio_path):
    # 1. Load and preprocess audio
    audio, sr = librosa.load(audio_path, sr=16000)  # Whisper uses 16kHz
    
    # 2. Extract features
    inputs = processor(
        audio, 
        sampling_rate=sr, 
        return_tensors="np",  # Return NumPy arrays for ONNX
        padding=True,
        truncation=True
    )
    
    # 3. Prepare ONNX inputs
    ort_inputs = {
        "input_features": inputs.input_features.astype(np.float32),
        "decoder_input_ids": np.array([[processor.tokenizer.bos_token_id]], dtype=np.int64),
    }
    
    # 4. Run inference
    ort_outputs = sess.run(None, ort_inputs)
    logits = ort_outputs[0]  # First output contains logits
    
    # 5. Decode predictions
    predicted_ids = np.argmax(logits, axis=-1)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    return transcription

In [ ]:
# Usage
audio_path = "C:/Users/ifedi/Documents/Applied AI and ML/Session 2/INFO8665 - Projects in Machine Learning/SageCare-2.0/data-collection/unitedsynmed_small/audio/train/drug-brand-en-us-female-0d0979f3-3519-4e70-beae-339cda2b254a.wav"

result = transcribe_audio(audio_path)

print("Transcription:", result)